In [5]:
# Dependencies
from splinter import Browser
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import requests
import pymongo

In [6]:
# Setup splinter
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)

In [7]:
# Mars News URL to scrape
url = 'https://redplanetscience.com/'
browser.visit(url)
html = browser.html

In [8]:
# Create BeautifulSoup object; parse with 'html.parser'
soup = BeautifulSoup(html, 'html.parser')

In [9]:
# Examine the results and look for a ul with the class 'querylist'
result = soup.find('div', id="news",class_='container')

# Retrieve each list item
queries = result.find_all(class_='row')

In [10]:
for query in result:
    print(query)



<div class="col-md-12">
<hr/>
<div class="row">
<div class="col-md-4">
<div class="list_image">
<img src="https://mars.nasa.gov/system/news_items/list_view_images/8606_list_image.jpg"/>
</div>
</div>
<div class="col-md-8">
<div class="list_text">
<div class="list_date">October 6, 2022</div>
<div class="content_title">NASA Prepares for Moon and Mars With New Addition to Its Deep Space Network</div>
<div class="article_teaser_body">Robotic spacecraft will be able to communicate with the dish using radio waves and lasers.</div>
</div>
</div>
</div>
</div>


<div class="col-md-12">
<hr/>
<div class="row">
<div class="col-md-4">
<div class="list_image">
<img src="https://mars.nasa.gov/system/news_items/list_view_images/8554_PIA23553-320x240.jpg"/>
</div>
</div>
<div class="col-md-8">
<div class="list_text">
<div class="list_date">October 6, 2022</div>
<div class="content_title">Two of a Space Kind: Apollo 12 and Mars 2020</div>
<div class="article_teaser_body">Apollo 12 and the upcoming M

In [11]:
# Create empty arrays
titles = []
previews = []

# Loop through returned queries
for query in queries:
    try:
        # Retrieve the query title
        title_search = query.find('div', class_='content_title')

        # Access the query's text content and add to array
        title_text = title_search.text
        titles.append(title_text)
        # Access the query with CSS selectors, get text and add to array
        preview_search = query.find('div', class_='article_teaser_body')
        preview_text = preview_search.text   
        previews.append(preview_text) 

        print('\n-----------------\n')
        print(title_text)
        print(preview_text)

    except Exception as E:
        print(E) 




-----------------

NASA Prepares for Moon and Mars With New Addition to Its Deep Space Network
Robotic spacecraft will be able to communicate with the dish using radio waves and lasers.

-----------------

Two of a Space Kind: Apollo 12 and Mars 2020
Apollo 12 and the upcoming Mars 2020 mission may be separated by half a century, but they share several goals unique in the annals of space exploration.

-----------------

Virginia Middle School Student Earns Honor of Naming NASA's Next Mars Rover
NASA chose a seventh-grader from Virginia as winner of the agency's "Name the Rover" essay contest. Alexander Mather's entry for "Perseverance" was voted tops among 28,000 entries. 

-----------------

NASA Invites Students to Name Mars 2020 Rover
Through Nov. 1, K-12 students in the U.S. are encouraged to enter an essay contest to name NASA's next Mars rover.

-----------------

Meet the People Behind NASA's Perseverance Rover
These are the scientists and engineers who built NASA's next Mars r

In [13]:
browser.quit()

In [17]:
# Create dictionary to hold articles
article = [{'title':i, 'preview':j} for i,j in zip(titles, previews)]
print(article)

[{'title': 'NASA Prepares for Moon and Mars With New Addition to Its Deep Space Network', 'preview': 'Robotic spacecraft will be able to communicate with the dish using radio waves and lasers.'}, {'title': 'Two of a Space Kind: Apollo 12 and Mars 2020', 'preview': 'Apollo 12 and the upcoming Mars 2020 mission may be separated by half a century, but they share several goals unique in the annals of space exploration.'}, {'title': "Virginia Middle School Student Earns Honor of Naming NASA's Next Mars Rover", 'preview': 'NASA chose a seventh-grader from Virginia as winner of the agency\'s "Name the Rover" essay contest. Alexander Mather\'s entry for "Perseverance" was voted tops among 28,000 entries. '}, {'title': 'NASA Invites Students to Name Mars 2020 Rover', 'preview': "Through Nov. 1, K-12 students in the U.S. are encouraged to enter an essay contest to name NASA's next Mars rover."}, {'title': "Meet the People Behind NASA's Perseverance Rover", 'preview': "These are the scientists an

In [18]:
# Initialise PyMongo to work with MongoDBs
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

# Declare the Mars database
mars_data_db = client["mars_data_db"]
article_collection = mars_data_db["news"]

In [19]:
#Upload data into MongoDB
for article in article:
    article_collection.insert_one(article)